In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv("train.csv", index_col= 0)
display(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
display(df.shape)

In [ ]:
display(df.columns)

In [ ]:
columns_to_select = [
    'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
    'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 
    'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 
    'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 
    'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd'
]

# Select columns
df_selected = df[columns_to_select]

In [ ]:
display(df_selected.head(5))

In [ ]:
df_selected.columns

In [ ]:
# Missing values
missing_values = df_selected.isnull().sum()

# Duplicate rows
duplicates = df_selected.duplicated().sum()

display(missing_values)
display(duplicates)

In [ ]:
# Print all dupes, originals and duplicates
duplicates = df_selected[df_selected.duplicated(keep=False)]
display(duplicates)

In [ ]:
#Sort Duplicates by Living Area in order to see which pairs are dupes

duplicates_sorted = duplicates.sort_values(by='GrLivArea')
display(duplicates_sorted)

In [ ]:
# Examine the Missing Values

df_with_nans = df_selected[df_selected.isna().any(axis=1)]
display(df_with_nans)

In [ ]:
# House with no Basements are coded as Nan

# I replace this with "no basement" to avoid confusion

# Select all columns starting with 'Bsmt'
bsmt_columns = df_selected.columns[df_selected.columns.str.startswith('Bsmt')]

# Replace both NaN and <NA> with 'no basement'
df_selected.loc[:, bsmt_columns] = df_selected.loc[:, bsmt_columns].apply(lambda x: x.fillna('no basement').replace({np.nan: 'no basement'}))

#df_selected.loc[:, df_selected.columns.str.startswith('Bsmt')] = df_selected.loc[:, df_selected.columns.str.startswith('Bsmt')].fillna('no basement')

#df_selected.loc[:, df_selected.columns.str.startswith('Bsmt')] = df_selected.loc[:, df_selected.columns.str.startswith('Bsmt')].replace({np.nan: 'no basement'})



In [ ]:
df_selected['BsmtQual'].unique()

In [ ]:
missing_values = df_selected.isnull().sum()
display(missing_values)

In [ ]:
# NaN in the "Electrical" column

pd.set_option('display.max_columns', 27)
missing_electrical = df_selected[df_selected['Electrical'].isna()]
display(missing_electrical)

In [ ]:
# There is one missing value in Electricity
# Instead of just finding the mode of the electricity variable, I take the mode
# of Electricity for all of the houses that have similar characteristics in terms of
# Central Air, Heating, and Heating QC.

filtered_data = df_selected[(df_selected['CentralAir'] == 'Y') & 
                            (df_selected['Heating'] == 'GasA') & 
                            (df_selected['HeatingQC'] == 'Gd')]

# Mode
mode_electrical = filtered_data['Electrical'].mode()
print(filtered_data.head())

# Display the mode(s)
display(mode_electrical)

In [210]:
# replace the one missing value with the associated mode from above
df_selected.loc[:, 'Electrical'] = df_selected['Electrical'].fillna('SBrkr')

In [211]:
missing_values = df_selected.isnull().sum()
display(missing_values)

## No more Missing values

ExterCond       0
Foundation      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinSF1      0
BsmtFinType2    0
BsmtFinSF2      0
BsmtUnfSF       0
TotalBsmtSF     0
Heating         0
HeatingQC       0
CentralAir      0
Electrical      0
1stFlrSF        0
2ndFlrSF        0
LowQualFinSF    0
GrLivArea       0
BsmtFullBath    0
BsmtHalfBath    0
FullBath        0
HalfBath        0
BedroomAbvGr    0
KitchenAbvGr    0
KitchenQual     0
TotRmsAbvGrd    0
dtype: int64

In [212]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 1 to 1460
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ExterCond     1460 non-null   object
 1   Foundation    1460 non-null   object
 2   BsmtQual      1460 non-null   object
 3   BsmtCond      1460 non-null   object
 4   BsmtExposure  1460 non-null   object
 5   BsmtFinType1  1460 non-null   object
 6   BsmtFinSF1    1460 non-null   int64 
 7   BsmtFinType2  1460 non-null   object
 8   BsmtFinSF2    1460 non-null   int64 
 9   BsmtUnfSF     1460 non-null   int64 
 10  TotalBsmtSF   1460 non-null   int64 
 11  Heating       1460 non-null   object
 12  HeatingQC     1460 non-null   object
 13  CentralAir    1460 non-null   object
 14  Electrical    1460 non-null   object
 15  1stFlrSF      1460 non-null   int64 
 16  2ndFlrSF      1460 non-null   int64 
 17  LowQualFinSF  1460 non-null   int64 
 18  GrLivArea     1460 non-null   int64 
 19  BsmtFullBat

# Above has replaced the missing values and Identified potential duplicates

# Convert Categorical and Ordinal Variables to Numeric

## Create Lists of Categorical, Ordinal, and Numeric Variables'

In [213]:
cat_vars = ['Foundation', 'Heating', 'Electrical']

ordinal_vars = ['ExterCond', 'BsmtQual', 'BsmtCond' 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC',
                'CentralAir', 'KitchenQual']                

numeric_vars = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
                'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'Bedroom', 'Kitchen', 'TotRmsAbvGrd']

## Create dictionary of variable codings

In [214]:
var_dict = {
    'ExterCond': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'BsmtQual': ['no basement', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'BsmtCond': ['no basement', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'BsmtExposure': ['no basement', 'No', 'Mn', 'Av', 'Gd'],
    'BsmtFinType1': ['no basement', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
    'BsmtFinType2': ['no basement', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
    'HeatingQC': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'CentralAir': ['N', 'Y'],
    'KitchenQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex']
}

In [215]:
 df_selected = df_selected.copy()

# Define binary variables
binary_vars = ['CentralAir']

def create_mapping(categories, variable):
    mapping = {}
    
    # Handle binary variables
    if variable in binary_vars:
        mapping = {'N': 0, 'Y': 1}
        return mapping
    
    # Handle ordinal variables
    start_idx = 0
    if categories[0] == 'no basement':
        mapping['no basement'] = 0
        start_idx = 1
    
    for i, category in enumerate(categories[start_idx:], start=1):
        mapping[category] = i
        
    return mapping

# Debugging function to find mismatched values
def find_mismatches(df, variable, mapping):
    actual_values = set(df[variable].unique())
    mapping_keys = set(mapping.keys())
    mismatched_values = actual_values - mapping_keys
    return mismatched_values

# Create all mappings and apply them to the dataframe
for variable, categories in var_dict.items():
    # Create mapping for this variable
    mapping = create_mapping(categories, variable)
    
    # Find mismatched values
    mismatches = find_mismatches(df_selected, variable, mapping)
    if mismatches:
        print(f"Warning: Mismatched values for '{variable}': {mismatches}")
    
    # Apply mapping using .map()
    df_selected[variable] = df_selected[variable].map(mapping)
    
    # Handle potential NaN values
    if df_selected[variable].isna().any():
        print(f"Warning: Found NaN values in '{variable}' after mapping. Filling with 0.")
        df_selected[variable] = df_selected[variable].fillna(0)
    
    # Convert to int64
    df_selected[variable] = df_selected[variable].astype('int64')
    
    # Print results for verification
    print(f"\nProcessed {variable}:")
    print("Mapping:", mapping)
    print("Unique values in DataFrame:", df_selected[variable].unique())


Processed ExterCond:
Mapping: {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
Unique values in DataFrame: [3 4 2 1 5]

Processed BsmtQual:
Mapping: {'no basement': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
Unique values in DataFrame: [4 3 5 0 2]

Processed BsmtCond:
Mapping: {'no basement': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
Unique values in DataFrame: [3 4 0 2 1]

Processed BsmtExposure:
Mapping: {'no basement': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4}
Unique values in DataFrame: [1 4 2 3 0]

Processed BsmtFinType1:
Mapping: {'no basement': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}
Unique values in DataFrame: [6 5 1 3 4 0 2]

Processed BsmtFinType2:
Mapping: {'no basement': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}
Unique values in DataFrame: [1 4 0 5 3 2 6]

Processed HeatingQC:
Mapping: {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
Unique values in DataFrame: [5 4 3 2 1]

Processed CentralAir:
Mapping: {'N': 0, 'Y': 1}
Unique

In [216]:
df_selected.isnull().sum()

ExterCond       0
Foundation      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinSF1      0
BsmtFinType2    0
BsmtFinSF2      0
BsmtUnfSF       0
TotalBsmtSF     0
Heating         0
HeatingQC       0
CentralAir      0
Electrical      0
1stFlrSF        0
2ndFlrSF        0
LowQualFinSF    0
GrLivArea       0
BsmtFullBath    0
BsmtHalfBath    0
FullBath        0
HalfBath        0
BedroomAbvGr    0
KitchenAbvGr    0
KitchenQual     0
TotRmsAbvGrd    0
dtype: int64

In [217]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 1 to 1460
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ExterCond     1460 non-null   int64 
 1   Foundation    1460 non-null   object
 2   BsmtQual      1460 non-null   int64 
 3   BsmtCond      1460 non-null   int64 
 4   BsmtExposure  1460 non-null   int64 
 5   BsmtFinType1  1460 non-null   int64 
 6   BsmtFinSF1    1460 non-null   int64 
 7   BsmtFinType2  1460 non-null   int64 
 8   BsmtFinSF2    1460 non-null   int64 
 9   BsmtUnfSF     1460 non-null   int64 
 10  TotalBsmtSF   1460 non-null   int64 
 11  Heating       1460 non-null   object
 12  HeatingQC     1460 non-null   int64 
 13  CentralAir    1460 non-null   int64 
 14  Electrical    1460 non-null   object
 15  1stFlrSF      1460 non-null   int64 
 16  2ndFlrSF      1460 non-null   int64 
 17  LowQualFinSF  1460 non-null   int64 
 18  GrLivArea     1460 non-null   int64 
 19  BsmtFullBat

# Encode Categorical Variables Using the One Hot Encoder

In [218]:
# Create and fit the encoder
encoder = OneHotEncoder(sparse_output=False, drop=None, handle_unknown='error')
encoded_cats = encoder.fit_transform(df_selected[cat_vars])

# Get feature names and create DataFrame with boolean type
feature_names = encoder.get_feature_names_out(cat_vars)
encoded_df = pd.DataFrame(encoded_cats, columns=feature_names, index=df.index).astype(bool)

# Drop original categorical columns and merge with encoded ones
df_selected = df_selected.drop(columns=cat_vars)
df_final = pd.concat([df_selected, encoded_df], axis=1)

#change CentraAir to boolean
df_final['CentralAir'] = df_final['CentralAir'].astype(bool)

# Verification prints
print("Original df_selected shape:", df_selected.shape)
print("encoded_df shape:", encoded_df.shape)
print("Final merged df shape:", df_final.shape)
print("\nColumn types in final dataframe:")
print(df_final.dtypes.value_counts())

Original df_selected shape: (1460, 24)
encoded_df shape: (1460, 17)
Final merged df shape: (1460, 41)

Column types in final dataframe:
int64    23
bool     18
Name: count, dtype: int64


In [219]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 1 to 1460
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   ExterCond          1460 non-null   int64
 1   BsmtQual           1460 non-null   int64
 2   BsmtCond           1460 non-null   int64
 3   BsmtExposure       1460 non-null   int64
 4   BsmtFinType1       1460 non-null   int64
 5   BsmtFinSF1         1460 non-null   int64
 6   BsmtFinType2       1460 non-null   int64
 7   BsmtFinSF2         1460 non-null   int64
 8   BsmtUnfSF          1460 non-null   int64
 9   TotalBsmtSF        1460 non-null   int64
 10  HeatingQC          1460 non-null   int64
 11  CentralAir         1460 non-null   bool 
 12  1stFlrSF           1460 non-null   int64
 13  2ndFlrSF           1460 non-null   int64
 14  LowQualFinSF       1460 non-null   int64
 15  GrLivArea          1460 non-null   int64
 16  BsmtFullBath       1460 non-null   int64
 17  BsmtHalfBath       

# No Missing Values, All Numeric variables

# Git Code

In [224]:
!git status

On branch Data_Inspection_Vars_28_54
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Data_Inspection_Vars_28_54.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [225]:
!git add Data_Inspection_Vars_28_54.ipynb

In [226]:
!git commit -m "Updated FORMULAS for data inspection notebook with changes to variables 28-54"

[Data_Inspection_Vars_28_54 5c74423] Updated FORMULAS for data inspection notebook with changes to variables 28-54
 1 file changed, 28 insertions(+), 2270 deletions(-)


In [227]:
!git push origin Data_Inspection_Vars_28_54

To https://github.com/AaronMartin78/Kaggle-Competition-Housing-Prices-I
   c0219c1..5c74423  Data_Inspection_Vars_28_54 -> Data_Inspection_Vars_28_54
